In [1]:
import sys
sys.path.append('../')
import env

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
READ_FILE = 'Origin'

In [54]:
def main(args, array):
    df = pd.read_csv(env.get_full_path(READ_FILE, args),
                     names=env.get_col_names(),
                     encoding='Shift_JISx0213')
    array[args.dir][args.ratio][args.seed][args.csv] = df.copy()

csv_array = {}
env.for_default_init(main, csv_array, csv=['od'])

In [55]:
people='p10000'
ratio='r4'
seed='s123'
csv='od'
df = csv_array[people][ratio][seed][csv].copy()

In [61]:
df.head()

,c00,c01,c02,c03,c04,c05,c06,c07,c08,c09,...,c20,c21,c22,c23,c24,c25,c26,c27,c28,c29
0,15131,Vehicle,FALSE,国道8号p(census)@389,国道8号o(census)@419,国道8号n(census)@503,石川県道198号a(census)@884,石川県道198号b(census)@887,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12929,Vehicle,FALSE,石川県道17号b(census)@430,石川県道8号l(census)@439,石川県道8号k(census)@543,石川県道8号j(census)@587,石川県道8号i(census)@722,石川県道291号b(census)@959,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15542,Vehicle,FALSE,石川県道196号k(census)@908,石川県道196号e(census)@1099,,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19899,Vehicle,FALSE,石川県道8号s(census)@994,,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16531,Vehicle,FALSE,石川県道196号d(census)@753,国道8号h(census)@800,国道8号i(census)@858,国道8号j(census)@873,国道8号k(census)@952,国道8号l(census)@1008,国道8号m(census)@1043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def create_2d_dataframe():
    df = np.zeros((env.MAX_TIME_COUNT() * env.MAX_AREA_COUNT(), 3))
    df = pd.DataFrame(df, columns=['time', 'area', 'people'])

    index = 0
    for _times in env.TIMES_LIST():
        for area in range(env.MAX_AREA_COUNT()):
            df.loc[index, ['time', 'area']] = [_times, area]
            index += 1

    return df

In [8]:
def interpolate_time(time):
    time = int(time)
    times_list = [3600 * (i + 1) for i in range(env.MAX_TIME_COUNT())]
    times = []

    if time in times_list:
        times = time
    elif 0 < time < times_list[0]:
        times = times_list[0]
    elif times_list[0] < time < times_list[1]:
        times = times_list[1]
    elif times_list[1] < time < times_list[2]:
        times = times_list[2]
    elif times_list[2] < time < times_list[3]:
        times = times_list[3]
    elif times_list[3] < time < times_list[4]:
        times = times_list[4]
    elif times_list[4] < time < times_list[5]:
        times = times_list[5]

    return str(times)

In [9]:
def func_road_to_area(args):
    df = pd.read_csv(env.get_full_path('include_area_-1', args),
                     encoding='Shift_JISx0213')
    df = df.loc[:, ['road', 'area']]
    for row in np.asanyarray(df):
        road_to_area[row[0]] = float(row[1])

    road_to_area[np.nan] = np.nan
    
road_to_area = {}
env.for_default(func_road_to_area, csv=['census'])

In [10]:
def count_per_time(series):
    row = series.values.tolist()
    row = row[3:]

    for i in range(len(row)):
        # 消滅する前にスペースが入っているので
        if row[i] == ' ':
            break
        pass_time = row[i].split('@')[1]
        pass_time = interpolate_time(pass_time)
        
        if (i >= 3) and (type(row[i]) is str) and ('(census)' in row[i]):
            road_name = row[i].split('(census)')[0]
            _area = road_to_area[road_name]
            df_base.loc[(df_base['time'] == pass_time) & (df_base['area'] == _area), ['people']] += 1
            
    return series

In [29]:
df_base = create_2d_dataframe()
df.apply(lambda x: count_per_time(x), axis=1)

df_group = df_base.groupby(['time'])
for _name, _group in df_group:
    args = env.ARGS_FOR_LIST('p10000', 'r4', 's123', 'od')
    _group.to_csv(env.get_full_path('2D_per_time', args, any=_name), index=False)

In [12]:
# df_base.to_csv('/Users/kessapassa/OneDrive/research_log/20190803/2D_per_time/p10000r4s123od_3600.csv', index=False)

In [13]:
# df.applymap(lambda x: (x.split('(census)')[0]) if (type(x) is str) and ('(census)' in x) else x)

In [14]:
# import re
# pattern = '(.*)\(census\)(.*)@(\d+)'

# series = df.loc[0].copy()
# row = series.values.tolist()
# for i in range(len(row)):
#     if (i >= 3) and (type(row[i]) is str) and ('(census)' in row[i]):
#                 result = re.compile(pattern).match(row[i])
#                 print(result[1], result[2])
# #         road_name = row[i].split('(census)')[0]
# #         repatter = re.compile(pattern)
# #         result = repatter.match(row[i])

In [15]:
def func_road_to_area(args):
    df = pd.read_csv(env.get_full_path('include_area_-1', args),
                     encoding='Shift_JISx0213')
    df = df.loc[:, ['road', 'area']]
    for row in np.asanyarray(df):
        road_to_area[row[0]] = float(row[1])

    road_to_area[np.nan] = np.nan
    
road_to_area = {}
env.for_default(func_road_to_area, csv=['census'])

In [16]:
import re
pattern = '(.*)\(census\)(.*)@(\d+)'
def extract_only_area(series):
    row = series.values.tolist()
    df_new = row[0:3]
    df_base = []

    for i in range(len(row)):
        if (i >= 3) and (type(row[i]) is str) and ('(census)' in row[i]):
            road_name = row[i].split('(census)')[0]
            _area = road_to_area[road_name]
            df_base.append(_area)
            
            result = re.compile(pattern).match(row[i])
            road_name1 = result[1]
            time = result[2]
#             print(road_name1, time)

    df_new.extend(df_base)

    return pd.Series(df_new)

In [17]:
df = df.apply(lambda x: extract_only_area(x), axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,15131,Vehicle,FALSE,78.0,69.0,68.0,56.0,56.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12929,Vehicle,FALSE,56.0,56.0,47.0,37.0,37.0,19.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15542,Vehicle,FALSE,49.0,47.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19899,Vehicle,FALSE,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16531,Vehicle,FALSE,47.0,48.0,48.0,48.0,57.0,58.0,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_area = df.iloc[:, 3:]
df_area.head()

,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,78.0,69.0,68.0,56.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56.0,56.0,47.0,37.0,37.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47.0,48.0,48.0,48.0,57.0,58.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_area = df_area.applymap(lambda x: np.nan if x == -1 else x)

In [20]:
df_area = df_area.dropna(how='all')

In [21]:
def sort_head(series):
    row = series.values.tolist()
    if np.isnan(row[0]):
        row = np.array(row)
        row = row[~np.isnan(row)]
        
    return pd.Series(row)

In [22]:
df_area = df_area.apply(lambda x: sort_head(x), axis=1)
df_area

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,78.0,69.0,68.0,56.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56.0,56.0,47.0,37.0,37.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47.0,48.0,48.0,48.0,57.0,58.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,49.0,49.0,49.0,49.0,40.0,40.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,47.0,49.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,47.0,47.0,49.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,68.0,56.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,47.0,47.0,47.0,37.0,37.0,37.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
def remove_deculp_area(series):
    row = np.array(series)
    row = list(row[~np.isnan(row)])
    
    for i in range(len(row)-1):
        if (i < len(row)-1) and (row[i] == row[i+1]):
            row.pop(i)
            
    return pd.Series(row)

In [24]:
df_area = df_area.apply(lambda x: remove_deculp_area(x), axis=1)
df_area

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,78.0,69.0,68.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56.0,47.0,37.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47.0,48.0,48.0,57.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,49.0,49.0,40.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,47.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,47.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,68.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,47.0,47.0,37.0,37.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
counter=0
for series in df_area.values:
    for x in series:
        if not np.isnan(x):
            counter+=1

counter

11470

In [26]:
def extract_od_area(args, array):
    df = pd.read_csv(env.get_full_path(READ_FILE, args),
                     names=env.get_col_names(),
                     encoding='Shift_JISx0213')
    
    df = df.apply(lambda x: extract_only_area(x), axis=1)
    df_area = df.iloc[:, 3:]
    df_area = df_area.applymap(lambda x: np.nan if x == -1 else x)
    df_area = df_area.dropna(how='all')
    df_area = df_area.apply(lambda x: sort_head(x), axis=1)
    df_area = df_area.apply(lambda x: remove_deculp_area(x), axis=1)
    
    array[args.dir][args.ratio][args.seed][args.csv] = df_area.copy()

od_array = {}
env.for_default_init(extract_od_area, od_array, ['od'])

In [27]:
def count_sum(args):
    df = od_array[args.dir][args.ratio][args.seed][args.csv].copy()
    
    counter=0
    for series in df.values:
        for x in series:
            if not np.isnan(x):
                counter+=1
                
    print(args.dir, args.ratio, args.seed)
    print(counter)
    print('\n')
    
env.for_default(count_sum, ['od'])

p10000 r4 s123
11470


p10000 r5 s123
14454


p10000 r6 s123
17538


p20000 r4 s123
23110


p20000 r5 s123
28711


p20000 r6 s123
34801


